## Osher Dighorkar     
## Yuval Brunshtein   
## **Github Link**  : https://github.com/YuvalBru/Web_Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import calendar
import json

### In the following function we extract the links for all of the Renu type cars (1)

In [2]:
base_url = 'https://www.ad.co.il'


def get_all_renu_cars_linkss():
    renu_base_url = f"{base_url}/category/3?sp261=13894&pageindex=%page_num%"
    cars_urls = []

    current_page_number = 1

    while True:
        current_renu_page_url = renu_base_url.replace("%page_num%", str(current_page_number))
        response = requests.get(current_renu_page_url)

        soup = BeautifulSoup(response.content, 'html.parser')
        target_divs = soup.find_all('div', class_='card-body p-md-3')

        new_cars_urls = []
        for div in target_divs:
            a_tags = div.find('a', recursive=False)
            if a_tags is None:
                break

            partial_renu_car_url = a_tags.get("href")

            new_cars_urls.append(f"{base_url}{partial_renu_car_url}")
        if not new_cars_urls:
            break

        cars_urls.extend(new_cars_urls)
        current_page_number += 1

    return cars_urls

### In the following function we extract data from url of car ads. (2)

In [3]:
def get_data_from_renu_car_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    data_dict = {}
    table = soup.find('table', class_='table table-sm mb-4')
    desc = soup.find('meta', attrs={'name': 'description'})
    description_content = desc['content'] if desc else np.nan
    model_f = soup.find('h2', class_='card-title')
    model_ff = model_f.get_text(strip=True) if model_f else np.nan  
    price_ff = soup.find('div', class_ ="d-flex justify-content-between")
    if price_ff:
        price_f = price_ff.find_all('h2' , class_ = 'card-title')
        if len(price_f) == 2:
            fin_price = price_f[1].get_text(strip = True)
        else:
            fin_price = None
    imgs = soup.find('div' , class_ = 'row hide-on-mobile')
    if imgs:
        img_elements = imgs.find_all('img', class_='desktop-thumbnail bg-video') 
        num_imgs = len(img_elements)
    else:
        num_imgs = 0
    manafacturer = re.findall('רנו',model_ff )
    dates_cre = soup.find_all('div' , class_ = 'px-3')
    reg = r'\d{2}/\d{2}/\d{4}'
    date_cre_fin = re.search(reg, dates_cre[0].get_text(strip = True))[0]
    date_repub_fin = re.search(reg, dates_cre[1].get_text(strip = True))[0]
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        key = columns[0].get_text(strip=True)
        value = columns[1].get_text(strip=True)
        data_dict[key] = value
 
    return data_dict, description_content, model_ff, manafacturer[0], fin_price, num_imgs, date_cre_fin, date_repub_fin

### In the following function we extract dates for test from description to the best of our abillities, By examinning the datawe have come to the realization that a lot of people don't write test in the table of the ad but in the description.To tackle the problem we create a function that extracts dates from the descrpition but due to the uniquely and unevenly distribution of ways which people write the date of their test we use regular expressions (based on the most common types of ways to write test) to extract the dates to the best of our abillity in order to capture most dates which are written in the description. (3)


In [4]:

def calculate_days_to_test(description,date_incase):
    year_pattern = r'טסט לשנה'
    month_pattern = r'טסט לחודש'
    till_pattern = r'טסט עד'
    till_pattern2 = r'טסט עד לחודש'
    specific_date_pattern = r'טסט הבא ב([א-ת]+ \d{4})'
    date_patterns = [
        (r'(\d{2}/\d{2}/\d{4})', '%d/%m/%Y'),
        (r'(\d{2}/\d{2}/\d{2})', '%d/%m/%y'),
        (r'(\d{2}-\d{2}-\d{4})', '%d-%m-%Y'),
        (r'(\d{2}-\d{2}-\d{2})', '%d-%m-%y'),
        (r'(\d{1,2}/\d{2})', '%m/%y'),
        (r'(\d{1,2}-\d{2})', '%m-%y')
        
    ]
    hebrew_patterns = [
    r'ינואר',
    r'פברואר',
    r'מרץ',
    r'אפריל',
    r'מאי',
    r'יוני',
    r'יולי',
    r'אוגוסט',
    r'ספטמבר',
    r'אוקטובר',
    r'נובמבר',
    r'דצמבר'
    ]

    today = datetime.today()
    next_test_date = None
    if re.search(year_pattern, description):
        next_test_date = date_incase + timedelta(days=365)
    
    elif re.search(month_pattern, description):
        next_test_date = date_incase + timedelta(days=30)

    elif re.search(till_pattern , description):
       
        for pattern, date_format in date_patterns:
            match = re.search(pattern, description)
            if match:
                date_str = match.group(1)
                next_test_date = datetime.strptime(date_str, date_format)
                if date_format == '%m/%y' or date_format == '%m-%y':
                    next_test_date = next_test_date.replace(year=2000 + next_test_date.year % 100)
                    last_day = calendar.monthrange(next_test_date.year, next_test_date.month)[1]
                    next_test_date = next_test_date.replace(day = last_day)
                break
            else:
                match = re.search(specific_date_pattern, description)
                match2 = re.search(till_pattern2 , description)
            if match:
                hebrew_date_str = match.group(1)
                month_names = {
                    'ינואר': 1, 'פברואר': 2, 'מרץ': 3, 'אפריל': 4, 'מאי': 5, 'יוני': 6,
                    'יולי': 7, 'אוגוסט': 8, 'ספטמבר': 9, 'אוקטובר': 10, 'נובמבר': 11, 'דצמבר': 12
                }
                month_name, year = hebrew_date_str.split()
                month = month_names[month_name]
                year = int(year)  
                place_holder, last_day = calendar.monthrange(year, month)
                last_day_date = datetime(year, month, last_day)
                next_test_date = datetime(year, month, last_day)
            elif match2:
                month_names = {
                    'ינואר': 1, 'פברואר': 2, 'מרץ': 3, 'אפריל': 4, 'מאי': 5, 'יוני': 6,
                    'יולי': 7, 'אוגוסט': 8, 'ספטמבר': 9, 'אוקטובר': 10, 'נובמבר': 11, 'דצמבר': 12
                }
                for pattern in hebrew_patterns:
                    match3 = re.search(pattern, description)
                    if match3:
                        month = month_names[match3[0]]
                        place_holder, last_day = calendar.monthrange(date_incase.year, month)
                        last_day_date = datetime(date_incase.year, month, last_day)
                        next_test_date = datetime(date_incase.year, month, last_day)
                        
    if next_test_date:
        days_to_test = (next_test_date - today).days
        days_to_test = (next_test_date - today).days
        return days_to_test  
    else: 
        return None

### In the following function we take the data from (1) and organize it, the data comes in the format of the website which in Hebrew, we then move our data into a third dictionary (when the first dictionary is from (1) the second is in order to translate into English) and make it into a pandas dataframe. (4)

In [5]:
def organize_url_translate(url):
    ht_content, ht_content['Description'], ht_content['Model'], ht_content['Manufactor'], ht_content['Price'], ht_content['Pic_num'], ht_content['Cre_date'], ht_content['Repub_date'] = get_data_from_renu_car_url(url)
    hebrew_to_key = {
        'שנה': 'Year',
        'יד': 'Hand',
        'ת. הילוכים': 'Gear',
        'נפח': 'capacity_Engine',
        'סוג מנוע': 'Engine_type',
        'ק"מ': 'Km',
        'אזור': 'Area',
        'עיר': 'City' ,
        'Description' :'Description',
        'Model' : 'Model' ,
        'צבע' : 'Color' ,
        'Cre_date' : 'Cre_date' ,
        'Repub_date' : 'Repub_date' ,
        'טסט עד' : 'Test' , 
        'Pic_num' : 'Pic_num' ,
        'בעלות קודמת' : 'Prev_ownership' ,
        'בעלות נוכחית' : 'Curr_ownership' , 
        'Price' : 'Price' , 
        'Manufactor' : 'Manufactor'
    }

    data2 = {
        'Year': None,
        'Hand': None,
        'Gear': None,
        'capacity_Engine': None,
        'Engine_type': None,
        'Km': None,
        'Area': None,
        'City': None,
        'Description': None,
        'Model': None,
        'Color': None,
        'Cre_date': None,
        'Repub_date': None,
        'Test': None,
        'Pic_num': None,
        'Prev_ownership': None,
        'Curr_ownership': None,
        'Price' : None,
        'Manufactor' : None
    } 

    for hebrew_label, value in ht_content.items():
        if hebrew_label in hebrew_to_key:
            key = hebrew_to_key[hebrew_label]
            if key in ['Hand', 'Km', 'capacity_Engine','Test'  , 'Year'] and value != None:
                if (key == 'Test'):
                    test_date = datetime.strptime(value, '%m/%Y')
                    place_holder, last_day = calendar.monthrange(test_date.year, test_date.month)
                    last_day_date = datetime(test_date.year, test_date.month, last_day)
                    current_date = datetime.now()
                    difference = last_day_date - current_date
                    days_between = difference.days
                    value = str(days_between)
                
                value = int(value.replace(',', ''))
            elif key == 'Price' and value != None:
                value = float(value.replace(',','').replace('₪' , '').replace(' ' , ''))
            data2[key] = value
    if data2['Test'] == None and data2['Description'] != None:
        data2['Test'] = calculate_days_to_test(str(data2['Description']),datetime.strptime(data2['Cre_date'],'%d/%m/%Y'))
    data_finale = pd.DataFrame([data2])
    return data_finale

### In the following code we take the links from (1) and extract the data through (2) in order to filter our data into the desired year range in order to create an array of urls which are suitable for our purpose. (5)

In [6]:
renu_cars_in_between_years = []
min_year = 1980
max_year = 2015

renu_cars_links = get_all_renu_cars_linkss()

for car_link in renu_cars_links:
    data = get_data_from_renu_car_url(car_link)
    year = int(data[0]['שנה'])
    if min_year <= year <= max_year:
        renu_cars_in_between_years.append(car_link)

### In the following code we use (4) in order to extract data and turn it into a dataframe in a desirable manner. We then append each dataframe row in order to sum a large dataframe which consists all of the relevant data. (6)

In [7]:
dict_for_df = {
        'Year': None,
        'Hand': None,
        'Gear': None,
        'capacity_Engine': None,
        'Engine_type': None,
        'Km': None,
        'Area': None,
        'City': None,
        'Description': None,
        'Model': None,
        'Color': None,
        'Cre_date': None,
        'Repub_date': None,
        'Test': None,
        'Pic_num': None,
        'Prev_ownership': None,
        'Curr_ownership': None,
        'Price' : None,
        'Manufactor' : None
    }
df = pd.DataFrame(columns=dict_for_df.keys())
for item in renu_cars_in_between_years:
    df = pd.concat([df , organize_url_translate(item)], ignore_index = True)


### In the following code sectors we alter the dataframe made in (6) by changing  the types of the columns into our desirable types. As well as download the dataframe into a csv. (7)

In [8]:
categories = ["ציבורית", "ליסינג","פרטית"]
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'], categories=categories)

categories = ["דיזל", "חשמלי","גז","היבריד","בנזין"]
df['Engine_type'] = pd.Categorical(df['Engine_type'], categories=categories)

categories = ["פרטית","ליסינג","ציבורית"]
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'], categories=categories)

categories = ["רובוטית","ידנית","אוטומטית"]
df['Gear'] = pd.Categorical(df['Gear'], categories=categories)


In [9]:
integer_col = ['Year','Hand','capacity_Engine','Pic_num','Km','Test']
cat_col = ['Gear','Engine_type','Prev_ownership','Curr_ownership']
str_col = ['Area','City','Color','Description','Manufactor','Model']
df['Cre_date'] = df['Cre_date'].astype('datetime64')
df['Repub_date'] = df['Repub_date'].astype('datetime64')
df['Price'] = df['Price'].astype(float)
df[integer_col] = df[integer_col].astype('Int64')
df[cat_col] = df[cat_col].astype('category')
df[str_col] = df[str_col].astype(pd.StringDtype())

df['Cre_date'] = pd.to_datetime(df['Cre_date'])
df['Repub_date'] = pd.to_datetime(df['Repub_date'])
df.replace([pd.NaT, None], np.nan, inplace=True)


C:\Users\yuval\AppData\Local\Temp\ipykernel_3540\1645365920.py:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Cre_date'] = df['Cre_date'].astype('datetime64')
C:\Users\yuval\AppData\Local\Temp\ipykernel_3540\1645365920.py:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['Repub_date'] = df['Repub_date'].astype('datetime64')


In [10]:
df.to_csv('C:/Users/yuval/Downloads/Renu_osher_yuval_finale_web_scraping_Finale23.csv', index=False , encoding='utf-8-sig')
df

,Year,Hand,Gear,capacity_Engine,Engine_type,Km,Area,City,Description,Model,Color,Cre_date,Repub_date,Test,Pic_num,Prev_ownership,Curr_ownership,Price,Manufactor
0,2005,4,אוטומטית,1600,בנזין,220000,באר שבע והסביבה,באר שבע,רנו מגאן שנת 2005 מצב מכאני מעולה טסט ארוך עד ...,רנו מגאן II,כסף מטלי,2024-09-06,2024-09-06,235,4,פרטית,פרטית,4200.0,רנו
1,2015,2,אוטומטית,1500,דיזל,225000,זכרון - בנימינה,זכרון יעקב,"4 צמיגים חדשים, גאנטים מגנזיום 16 מצבר חדש של ...",רנו גרנד סניק,לבן,2024-09-06,2024-09-06,<NA>,6,פרטית,פרטית,25500.0,רנו
2,2013,1,אוטומטית,1200,חשמלי,70000,מושבים בשרון,שער אפרים,פלואנס חשמלי בטר פלייס יש לחדש סוללה וטסט,רנו פלואנס חשמלי,כחול כהה מטאלי,2024-04-25,2024-05-06,<NA>,4,פרטית,פרטית,25000.0,רנו
3,2008,3,אוטומטית,1600,דיזל,267000,באר שבע והסביבה,באר שבע,טסט עד 1.7 מטופלת,רנו מגאן II,כחול כהה,2023-12-24,2023-12-24,<NA>,2,פרטית,פרטית,5000.0,רנו
4,2014,2,אוטומטית,6000,בנזין,58,ירושלים והסביבה,ירושלים,רכב פצצה במצב מצויין חסכוני בטרוף !! שמור מאו...,רנו קליאו,אדום,2023-11-13,2023-11-13,82,5,פרטית,פרטית,30000.0,רנו
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2013,1,ידנית,1150,בנזין,<NA>,<NA>,ידידיה,את הרכב קניתי ישירות מהיבואן.,רנו קליאו,<NA>,2022-10-29,2022-10-29,<NA>,0,NaN,NaN,18000.0,רנו
68,2012,2,אוטומטית,90,חשמלי,42300,בית שמש והסביבה,בית שמש,"רכב חשמלי 100% שקט אמין נוסע 80 ק"" ללא דלק בכ...",רנו פלואנס חשמלי,<NA>,2022-09-20,2022-09-20,<NA>,0,NaN,NaN,27000.0,רנו
69,2015,3,אוטומטית,1197,בנזין,85000,ירושלים והסביבה,מבשרת ציון,"קפצור גרסה מעוצבת, בצבע אדום גג לבן. נמכר עקב ...",רנו קפצ`ור,<NA>,2022-08-27,2022-08-27,<NA>,0,NaN,NaN,44500.0,רנו
70,2014,2,אוטומטית,1461,NaN,<NA>,תל אביב,תל אביב יפו,<NA>,רנו 25,<NA>,2022-01-07,2022-01-07,<NA>,0,NaN,NaN,28000.0,רנו


### Supply Score part

### We take the  required data from the json format and build a dataframe we assume the aforementioned supply score refers to the amount of active cars per each model per each year because we assume that the supply refers to the amount of vehicles which are available.

In [11]:
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6'
response = requests.get(url)
relevant_info = []
if response.status_code == 200:
    data = response.json()
    for item in data['result']['records']:
        if item['tozar'] == 'רנו' and 1980 <= item['shnat_yitzur'] <= 2015:
            item_info = {
                'Manufactorer': item['tozar'],
                'Model': item['kinuy_mishari'],
                'Manufactured_Year': item['shnat_yitzur'],
                'Supply_Score': item['mispar_rechavim_pailim']
            }
            relevant_info.append(item_info)

    relevant_info = pd.DataFrame(relevant_info)
else:
    print(f"Request wasn't properly fulfilled, status code:{response.status_code}")

In [12]:
relevant_info['Supply_Score'] = relevant_info['Supply_Score'].astype('Int64') 


In [13]:
relevant_info.to_csv('C:/Users/yuval/Downloads/supply_score_Yuval_Osher_Finale.csv', index = False, encoding = 'utf-8-sig')
relevant_info

,Manufactorer,Model,Manufactured_Year,Supply_Score
0,רנו,THALIA,2003,10
1,רנו,MEGANE,2015,213
2,רנו,CLIO III,2013,78
3,רנו,מגאן TR 6.1 5 דלתות,1997,2
4,רנו,GEREND SCENIC,2007,3
...,...,...,...,...
1008,רנו,91 PH011 ITR 8.1 4 ד,1996,0
1009,רנו,MEGANE,2001,0
1010,רנו,MEGANE,1999,0
1011,רנו,FLUENCE Z.E.,2014,1
